In [1]:
import redis
from datetime import datetime
import socket
from IPython.display import clear_output






In [2]:



def watching_redis_publications(redis_obj, server_ip, item_id):
    global server_list
    
    with redis_obj.pipeline() as pipe:
        
        while True:
            try:
                # Get available inventory, watching for changes
                # related to this itemid before the transaction
                pipe.watch(item_id)
                server_status  = str(redis_obj.hget(item_id, b'server_status').decode())
                
                
                if server_status != 'busy':
                    server_list.append(server_ip)
                    print(f"Server is free {server_ip}")
                    pipe.unwatch()
                    break
                    
            except Exception as e:
                # Log total num. of errors by this user to buy this item,
                # then try the same process again of WATCH/HGET/MULTI/EXEC
                print(e)
    return None

    
    


    
def creating_connections(server_list): 
    '''
    Description:
        Create redis connection with servers
        
    Input:
        server_list  (list) : List of servers
        
    Output:
        server_connections  (dict) : Dictionary {server: redis_connection_obj, status} 
    
    
    '''
    server_connections = dict()
    
    for i in server_list:
        ip, port = i.split("_")
        server_connections[i] = [redis.Redis(host=ip, port=port)]
        
    return server_connections

In [ ]:
import time
import time
from threading import Thread
import redis
import random
import json


# Master Redis
r = redis.Redis(host='192.168.18.14', port=6379)
req_p = r.pubsub()

# subscribe to request Channel
req_p.subscribe('request')

# Server Setup
server_list = ['192.168.18.14_6379', '192.168.18.14_6378', '192.168.18.4_6379']
servers_obj = creating_connections(server_list)

# request list

req_p.get_message()
requests = []

while 1:
    
    
        try:
        
            request = req_p.get_message()['data']
            if request is not None and type(request) is not int:requests.append(json.loads(request))
        except TypeError as e:
            pass
            
        
        try:

            if len(server_list)  > 0 and  len(requests) > 0:
                # Poping Request and server who will process the request
                server = server_list.pop(0)
                request = requests.pop(0)

                # Getting redis object
                redis_obj = servers_obj[server][0]

                # Generating Random number
                item_id = int(random.random()*100000000)

                # Updating server Status
                request.update({"server_status":"busy"})

                # Storing request to the following db 
                redis_obj.hmset(item_id, request)
                redis_obj.publish('new_request', item_id)

                # Waiting for server to get free
                waiting_server_to_get_free = Thread(target= watching_redis_publications, args=(redis_obj, server, item_id ,))
                waiting_server_to_get_free.start()
            
            clear_output(wait=True)
            print(len(server_list) ,len(requests))
            print("All servers are busy or no request to entertain")




        except AttributeError as e:
            print(e)
            server_list.append(server)
            pass
        
    
        

3 0
All servers are busy or no request to entertain
